# Valorización de un Forward de BTU

## Configuración

Para ejecutar todos los ejemplos se debe importar la librería. Se sugiere utilizar siempre el alias `qcf`. 

In [1]:
import qcfinancial as qcf

Librerías adicionales.

In [2]:
import pandas as pd

Para formateo de `pandas.DataFrames`.

In [3]:
format_dict = {
    'nominal': '{0:,.2f}',
    'amort': '{0:,.2f}',
    'interes': '{0:,.2f}',
    'flujo': '{0:,.2f}',
    'amortizacion': '{0:,.2f}',
    'icp_inicial': '{0:,.2f}',
    'icp_final': '{0:,.2f}',
    'uf_inicial': '{0:,.2f}',
    'uf_final': '{0:,.2f}',
    'valor_tasa': '{0:,.4%}',
    'spread': '{0:,.4%}',
    'gearing': '{0:,.2f}',
    'amort_moneda_pago': '{0:,.2f}',
    'interes_moneda_pago': '{0:,.2f}',
    'valor_indice_fx': '{0:,.2f}'
}

## Construcción de la Curva

## Valorizar

Se da de alta un objeto `PresentValue`.

In [27]:
pv = qcf.PresentValue()

### Renta Fija Local

Se muestra el ejemplo de valorización de un bono bullet a tasa fija con las convenciones de la Bolsa de Comercio. Para el ejemplo usamos las características del BTU0150326.

Se dan de alta los parámetros requeridos para instanciar un objeto de tipo `FixedRateLeg`.

In [4]:
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(1, 3, 2015)
fecha_final = qcf.QCDate(1, 3, 2026)
bus_adj_rule = qcf.BusyAdjRules.NO
periodicidad = qcf.Tenor('6M')
periodo_irregular = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 100.0
amort_es_flujo = True
valor_tasa_fija = .015
tasa_cupon = qcf.QCInterestRate(
    valor_tasa_fija, 
    qcf.QC30360(),
    qcf.QCLinearWf()
)
moneda = qcf.QCCLP()
es_bono = True

# Se da de alta el objeto
pata_bono = qcf.LegFactory.build_bullet_fixed_rate_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    calendario,
    lag_pago,
    nominal,
    amort_es_flujo,
    tasa_cupon,
    moneda,
    es_bono
)

Se da de alta el valor de la TERA y luego se construye un objeto de tipo `ChileanFixedRateBond`.

In [5]:
tera = qcf.QCInterestRate(.015044, qcf.QCAct365(), qcf.QCCompoundWf())
bono_chileno = qcf.ChileanFixedRateBond(pata_bono, tera)

El Fwd de BTU se pacta a una TIR del 2.99% y vence el 9 de junio del 2023. Por lo tanto el valor a pagar (suponiendo que es un forward de compra) debe ser:

In [11]:
fecha_vcto = qcf.QCDate(9, 6, 2023)
tir = qcf.QCInterestRate(.0299, qcf.QCAct365(), qcf.QCCompoundWf())

valor_presente = bono_chileno.present_value(fecha_vcto, tir)
precio = bono_chileno.precio2(fecha_vcto, tir, 6)
valor_par = bono_chileno.valor_par(fecha_vcto)

print(f'Valor presente: {valor_presente:,.8f}')
print(f'Precio: {precio:,.4%}')
print(f'Valor par: {valor_par:,.18f}')

Valor presente: 96.58457614
Precio: 96.1903%
Valor par: 100.409932765513914887


Con esto el valor a pagar es:

In [12]:
nominal = 30_000
valor_uf = 35_956.37
valor_pago = nominal / 100.0 * precio * valor_par * valor_uf
print(f'Valor a pagar: {valor_pago:,.0f}')

Valor a pagar: 1,041,849,652


Para valorizar, debemos traer a valor presente este flujo en CLP con una tasa apropiada desde la fecha de vencimiento del forward hasta la fecha de valorización.

In [16]:
fecha_valorizacion = qcf.QCDate(17, 5, 2023)
plazo = fecha_valorizacion.day_diff(fecha_vcto)
tasa_descuento = .01 # Lineal Act/360

In [17]:
vp_tasa_contrato = valor_pago / (1 + tasa_descuento * plazo / 360)

Por otro lado, la TIR de mercado a fecha de valorización es 3.15%. Por lo que el valor presente del bono a fecha de valorización es:

In [19]:
tir_hoy = qcf.QCInterestRate(.0355, qcf.QCAct365(), qcf.QCCompoundWf())
valor_uf_hoy = 35_984.18
valor_presente_hoy = bono_chileno.present_value(fecha_valorizacion, tir_hoy)
precio_hoy = bono_chileno.precio2(fecha_valorizacion, tir_hoy, 6)
valor_par_hoy = bono_chileno.valor_par(fecha_valorizacion)
valor_hoy = nominal / 100.0 * precio_hoy * valor_par_hoy * valor_uf_hoy

In [20]:
m2m = valor_hoy - vp_tasa_contrato

In [23]:
print(f"Valor de mercado (M2M): {m2m:,.0f} CLP")

Valor de mercado (M2M): -15,783,877 CLP
